In [ ]:
# Import necessary libraries
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Path to the dataset on Google Drive
data_dir = '/content/drive/MyDrive/chest_xray'  # Update with the correct path in your Drive
train_dir = os.path.join(data_dir, 'train')
val_dir = os.path.join(data_dir, 'val')
test_dir = os.path.join(data_dir, 'test')

# Image data generators for preprocessing
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)
val_test_datagen = ImageDataGenerator(rescale=1.0/255.0)

# Create data generators
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

val_generator = val_test_datagen.flow_from_directory(
    val_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

test_generator = val_test_datagen.flow_from_directory(
    test_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary',
    shuffle=False
)

# Define the model
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator
)

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(test_generator)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

# Save the model
model.save('/content/drive/MyDrive/pneumonia_classifier.h5')

# Example: Predict on test images
import numpy as np
predictions = model.predict(test_generator)
predicted_classes = (predictions > 0.5).astype(int).flatten()

# Display some test results
for i in range(10):  # Show first 10 predictions
    print(f"Image {i+1}: {'PNEUMONIA' if predicted_classes[i] == 1 else 'NORMAL'}")


Mounted at /content/drive
Found 5216 images belonging to 2 classes.
Found 16 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


163/163 ━━━━━━━━━━━━━━━━━━━━ 578s 3s/step - accuracy: 0.7442 - loss: 0.5814 - val_accuracy: 0.5625 - val_loss: 0.9504
Epoch 2/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 363s 2s/step - accuracy: 0.8271 - loss: 0.3469 - val_accuracy: 0.6875 - val_loss: 0.6885
Epoch 3/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 364s 2s/step - accuracy: 0.8664 - loss: 0.3068 - val_accuracy: 0.7500 - val_loss: 0.6180
Epoch 4/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 353s 2s/step - accuracy: 0.8759 - loss: 0.2766 - val_accuracy: 0.8750 - val_loss: 0.3718
Epoch 5/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 390s 2s/step - accuracy: 0.8926 - loss: 0.2548 - val_accuracy: 0.8125 - val_loss: 0.4515
Epoch 6/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 371s 2s/step - accuracy: 0.9028 - loss: 0.2388 - val_accuracy: 0.7500 - val_loss: 0.5097
Epoch 7/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 386s 2s/step - accuracy: 0.9001 - loss: 0.2382 - val_accuracy: 0.7500 - val_loss: 0.8650
Epoch 8/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 351s 2s/step - accuracy: 0.8963 - loss: 0.2405 - val_accuracy: 0.687

Test Accuracy: 81.89%
20/20 ━━━━━━━━━━━━━━━━━━━━ 19s 851ms/step
Image 1: PNEUMONIA
Image 2: PNEUMONIA
Image 3: PNEUMONIA
Image 4: NORMAL
Image 5: NORMAL
Image 6: PNEUMONIA
Image 7: NORMAL
Image 8: NORMAL
Image 9: PNEUMONIA
Image 10: PNEUMONIA


In [ ]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Path to the dataset on Google Drive
data_dir = '/content/drive/Lung_Opacity'  # Updated with the new folder path

# Image data generators for preprocessing
data_gen = ImageDataGenerator(
    rescale=1.0/255.0,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # Splitting data into training and validation
)

# Create data generators
train_generator = data_gen.flow_from_directory(
    data_dir,
    target_size=(224, 224),  # Adjusted for ResNet50
    batch_size=32,
    class_mode='binary',
    subset='training'
)

val_generator = data_gen.flow_from_directory(
    data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='validation'
)

# Load ResNet50 as base model
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # Freeze base model layers

# Build the model
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator
)

# Save the model
model.save('/content/drive/MyDrive/lungopacity_resnet_classifier.h5')

# Example: Predict on images
import numpy as np
predictions = model.predict(val_generator)
predicted_classes = (predictions > 0.5).astype(int).flatten()

# Display some results
for i in range(10):  # Show first 10 predictions
    print(f"Image {i+1}: {'PNEUMONIA' if predicted_classes[i] == 1 else 'NORMAL'}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/Lung_Opacity'

In [ ]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
from google.colab import drive




# Path to the dataset on Google Drive
data_dir = 'converted_dataset'  # Updated with the new folder path

# Image data generators for preprocessing
data_gen = ImageDataGenerator(
    rescale=1.0/255.0,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # Splitting data into training and validation
)

# Create data generators
train_generator = data_gen.flow_from_directory(
    data_dir,
    target_size=(224, 224),  # Adjusted for ResNet50
    batch_size=32,
    class_mode='binary',
    subset='training'
)

val_generator = data_gen.flow_from_directory(
    data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='validation'
)

# Load ResNet50 as base model
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # Freeze base model layers

# Build the model
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator
)

# Save the model
model.save('/content/drive/MyDrive/lungopacity_resnet_classifier.h5')

# Example: Predict on images
import numpy as np
predictions = model.predict(val_generator)
predicted_classes = (predictions > 0.5).astype(int).flatten()

# Display some results
for i in range(10):  # Show first 10 predictions
    print(f"Image {i+1}: {'PNEUMONIA' if predicted_classes[i] == 1 else 'NORMAL'}")


Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.


ValueError: The PyDataset has length 0

In [ ]:

dataset_path = "/content/drive/MyDrive/Lung_Opacity"  # Ensure this matches your actual directory
print(os.listdir(dataset_path))


['785.jpg', '243.jpg', '572.jpg', '273.jpg', '623.jpg', '298.jpg', '392.jpg', '1068.jpg', '1036.jpg', '825.jpg', '915.jpg', '910.jpg', '616.jpg', '1035.jpg', '950.jpg', '184.jpg', '360.jpg', '402.jpg', '1046.jpg', '271.jpg', '1001.jpg', '202.jpg', '721.jpg', '575.jpg', '24.jpg', '451.jpg', '813.jpg', '253.jpg', '1091.jpg', '537.jpg', '848.jpg', '676.jpg', '495.jpg', '796.jpg', '400.jpg', '470.jpg', '725.jpg', '187.jpg', '170.jpg', '157.jpg', '827.jpg', '774.jpg', '307.jpg', '581.jpg', '951.jpg', '729.jpg', '881.jpg', '437.jpg', '1101.jpg', '482.jpg', '726.jpg', '14.jpg', '1019.jpg', '141.jpg', '67.jpg', '665.jpg', '514.jpg', '1026.jpg', '62.jpg', '521.jpg', '1037.jpg', '900.jpg', '50.jpg', '647.jpg', '596.jpg', '928.jpg', '819.jpg', '918.jpg', '970.jpg', '593.jpg', '508.jpg', '301.jpg', '111.jpg', '631.jpg', '985.jpg', '913.jpg', '1093.jpg', '673.jpg', '938.jpg', '323.jpg', '501.jpg', '46.jpg', '453.jpg', '618.jpg', '1073.jpg', '836.jpg', '1098.jpg', '232.jpg', '982.jpg', '989.jpg', '9